# Testing fastCPH

In [47]:
import numpy

In [48]:
import sklearn

In [49]:
import survwrap

In [50]:
#X, y = survwrap.load_test_data()
#X.shape, y.shape

In [51]:
my_data='flchain'
my_data_df=survwrap.datasets.get_data(my_data)

In [52]:
X, y= my_data_df.get_X_y()
X.shape, y.shape

((6524, 8), (6524,))

### Generate a (stratified) train-test split and Scale the features (only) 

First do the stratified splitting THEN do scaling, parameterized on X_train set ONLY 

In [53]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [54]:
X_train, X_test, y_train, y_test = survwrap.survival_train_test_split(X, y,rng_seed=2311)

In [55]:
scaler = StandardScaler().fit(X_train)
[X_train, X_test] = [ scaler.transform(_) for _ in  [X_train, X_test] ]
X_train.shape, X_test.shape

((4893, 8), (1631, 8))

In [56]:
#X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

balanced partitioning OK. Robst scaler damages the performance of DSM A LOT.
maybe did something wrong. It is standard scaler for now.

In [57]:
survwrap.get_indicator(y).sum(), survwrap.get_indicator(y_train).sum(), survwrap.get_indicator(y_test).sum(),


(1962, 1472, 490)

In [58]:
splitter = survwrap.survival_crossval_splitter(X_train,y_train,n_splits=3, n_repeats=2,rng_seed=2309)
print([ (survwrap.get_indicator(y_train[_[1]]).sum()) for _ in splitter])

[490, 491, 491, 490, 491, 491]


## check possible dimensionality reduction

In [59]:
from sklearn.decomposition import PCA

In [60]:
pca= PCA(n_components=0.995, random_state=2308).fit(X_train)
pca.n_components_

8

Only a modest dimensionality reduction is possible using PCA

In [61]:
## Stratified CV spliter for survival analysis

In [62]:
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold

In [63]:
#fastcph 
import lassonet
from lassonet import LassoNetCoxRegressor

In [76]:
fastcph=LassoNetCoxRegressor(tie_approximation='breslow',
                                hidden_dims=(8,8),
    lambda_start=0.001,
    path_multiplier=1.01,
    #gamma=0.5,
    #val_size=0.15,                        
    verbose=2,
                            )
fastcph

LassoNetCoxRegressor(device=device(type='cpu'), hidden_dims=(8, 8),
                     lambda_start=0.001, path_multiplier=1.01,
                     tie_approximation='breslow', val_size=0, verbose=2)

In [65]:
#X_train[:5]

In [71]:
#y_swap
#from pandas import DataFrame
#y_df= DataFrame({'time': survwrap.get_time(y_train),'event':survwrap.get_indicator(y_train)})
yup=numpy.column_stack((survwrap.get_time(y_train).astype('float32'),
                        survwrap.get_indicator(y_train).astype('float32')))
print(type(yup))
#yup=numpy.array(list(zip(survwrap.get_indicator(y_train).astype(float),survwrap.get_time(y_train).astype(float))))
yup[:10]
X_train.shape, yup.shape
#numpy.ndarray(y_train[1], y_train[0])

<class 'numpy.ndarray'>


((4893, 8), (4893, 2))

In [77]:
fitted_fastcph = fastcph.fit(X_train,yup)
fitted_fastcph

epoch: 0
loss: 10.413528442382812
epoch: 1
loss: 10.331826210021973
epoch: 2
loss: 10.26749324798584
epoch: 3
loss: 10.20324993133545
epoch: 4
loss: 10.139135360717773
epoch: 5
loss: 10.075178146362305
epoch: 6
loss: 10.011405944824219
epoch: 7
loss: 9.947855949401855
epoch: 8
loss: 9.884552955627441
epoch: 9
loss: 9.821542739868164
epoch: 10
loss: 9.758874893188477
epoch: 11
loss: 9.696599960327148
epoch: 12
loss: 9.634766578674316
epoch: 13
loss: 9.573421478271484
epoch: 14
loss: 9.51262378692627
epoch: 15
loss: 9.452433586120605
epoch: 16
loss: 9.392902374267578
epoch: 17
loss: 9.334098815917969
epoch: 18
loss: 9.276087760925293
epoch: 19
loss: 9.2189359664917
epoch: 20
loss: 9.162714004516602
epoch: 21
loss: 9.107474327087402
epoch: 22
loss: 9.053272247314453
epoch: 23
loss: 9.000205039978027
epoch: 24
loss: 8.948288917541504
epoch: 25
loss: 8.897614479064941
epoch: 26
loss: 8.848246574401855
epoch: 27
loss: 8.800251960754395
epoch: 28
loss: 8.753678321838379
epoch: 29
loss: 8.7085

LassoNetCoxRegressor(device=device(type='cpu'), hidden_dims=(8, 8),
                     lambda_start=0.001, path_multiplier=1.01,
                     tie_approximation='breslow', val_size=0, verbose=2)

In [ ]:
assert fastcph == fitted_fastcph

In [ ]:
# MACHECAZZO!
fastcph.predict(X_train)[:10], fastcph.predict(X_test)[:10]
#lassonet.plot_path(fastcph, fastcph.path, X_train, yup)

In [78]:
fastcph.path_


[HistoryItem(lambda_=0, state_dict=None, objective=7.603842735290527, loss=7.603842735290527, val_objective=7.6034722328186035, val_loss=7.6034722328186035, regularization=1.42737877368927, l2_regularization=tensor(4.6121), l2_regularization_skip=tensor(0.6328), selected=tensor([True, True, True, True, True, True, True, True]), n_iters=283),
 HistoryItem(lambda_=0.001, state_dict=None, objective=7.60376906645298, loss=7.602337837219238, val_objective=7.603562595963478, val_loss=7.602131366729736, regularization=1.4312292337417603, l2_regularization=tensor(4.6155), l2_regularization_skip=tensor(0.6364), selected=tensor([True, True, True, True, True, True, True, True]), n_iters=10),
 HistoryItem(lambda_=0.00101, state_dict=None, objective=7.6014807722342015, loss=7.600027084350586, val_objective=7.601235201097727, val_loss=7.599781513214111, regularization=1.4392949342727661, l2_regularization=tensor(4.6216), l2_regularization_skip=tensor(0.6436), selected=tensor([True, True, True, True,